In [ ]:
!pip install numba==0.55.1

In [ ]:
!wget "http://www.mgi.polymtl.ca/anjos/qaplib/data.d/lipa20a.dat"
!wget "http://www.mgi.polymtl.ca/anjos/qaplib/data.d/kra30a.dat"
!wget "http://www.mgi.polymtl.ca/anjos/qaplib/data.d/tai35b.dat"
!wget "http://www.mgi.polymtl.ca/anjos/qaplib/data.d/tho40.dat"

In [3]:
import numpy as np
import time
import random
import urllib
from numba import njit
from IPython.display import clear_output

In [4]:
@njit()
def _calculate_cost(chromosome, flow_matrix, distance_matrix):
  cost = 0
  n = chromosome.shape[0]
  for i in range(n):
    for j in range(n):
      cost += distance_matrix[i][j]*flow_matrix[chromosome[i]][chromosome[j]]
  return cost

@njit()
def _insertValues(child, parent, n, crossover_point):
    while(child.shape[0] != n):
      if(parent[crossover_point] not in child):
        child = np.append(child, parent[crossover_point])
      crossover_point = (crossover_point + 1) % n
    return child

@njit()
def _cutAndCrossfill(father,mother,n):
  
  crossover_point = random.randint(0, n)
  child1 = np.empty(n)
  child2 = np.empty(n)
  
  child1 = father[:crossover_point]
  child2 = mother[:crossover_point]
  child1 = _insertValues(child1,mother,n,crossover_point)
  child2 = _insertValues(child2,father,n,crossover_point)

  return child1,child2

In [5]:
class Individual:

  def __init__(self, size,chromosome):
    self.chromosome = chromosome
    self.size = size
    self.cost = np.inf

  def __str__(self):
    return str(self.chromosome)

  def __lt__(self,other):
    return self.cost < other.cost

  def __gt__(self,other):
    return self.cost > other.cost

  def __eq__(self,other):
    return self.cost == other.cost
  
  @staticmethod
  def generateRandom(n):
    return Individual(n,np.random.permutation(n))

  def mutation(self, mut_prob):
    if random.random() <= mut_prob:
      mp1 = random.randint(0, self.size - 1)
      mp2 = random.randint(0, self.size - 1)
      self.chromosome[mp1], self.chromosome[mp2] = self.chromosome[mp2], self.chromosome[mp1]

  def cutAndCrossfill(self,parent,cro_prob):
    
    if(random.random() > cro_prob):
      return Individual(self.size,self.chromosome), Individual(parent.size,parent.chromosome)
    
    n = self.size

    child1 = Individual(n,[])
    child2 = Individual(n,[])
    child1.chromosome,child2.chromosome = _cutAndCrossfill(self.chromosome,parent.chromosome,n)

    return (child1, child2)

  def calculate_cost(self,flow_matrix,distance_matrix):  
    self.cost = _calculate_cost(self.chromosome, flow_matrix,distance_matrix)


In [6]:
def bestTwoOutOfFive(population):
    # Select 5 individuals randomly
    tournament = [population[random.randint(
        0, len(population)-1)] for _ in range(len(population)//5)]

    # return best 2 of five
    return sorted(tournament)[:2]

In [7]:
def geneticAgorithm(f, d, pop_size, cro_prob, mut_prob, gen_size, off_size):
    """

    Genetic Algorithm for Quadratic Assignment Problem
    __________________________________________________
    
    f: Flow matrix (Numpy array of n x n dimentions)
    d: Distance matrix (Numpy array of n x n dimentions)
    pop_size: Population size (Integer)
    cro_prob: Crossover probability (Float [0.0,1.0))
    mut_prob: Mutation proability (Float [0.0,1.0))
    gen_size: Number of generations (Integer)
    off_size: Offspring size (Integer)
    """
    # Variables Initialization
    
    generations = 1
    n = len(f[0])
    offspring = []

    # INITIALISE population with random candidate solutions
    population = [Individual.generateRandom(n) for _ in range(pop_size)]

    best = min(population)
    # EVALUATE each candidate    
    for ind in population:
        ind.calculate_cost(f,d)

    # Repeat until termination condition is satisfied
    while(generations < gen_size + 1):

        for i in range(off_size//2):
            # PARENT SELECTION (best two out of five)
            father, mother = bestTwoOutOfFive(population)

            # CROSSOVER
            child1, child2 = father.cutAndCrossfill(mother, cro_prob)

            # MUTATION
            child1.mutation(mut_prob)
            child2.mutation(mut_prob)

            # EVALUATION OF NEW CANDIDATES
            child1.calculate_cost(f,d)
            child2.calculate_cost(f,d)

            offspring.append(child1)
            offspring.append(child2)
            
        # SURVIVOR SELECTION
        population = sorted(offspring)[:pop_size]
        offspring = []

        generations += 1

    return population


In [8]:
def read_qap_dataset(filename):
    def read_integers(filename):
      with open(filename) as f:
        return [int(elem) for elem in f.read().split()]

    file_it = iter(read_integers(filename))
    # Number of points
    n = next(file_it)
    # Distance between locations
    A = [[next(file_it) for j in range(n)] for i in range(n)]
    # Flow between factories
    B = [[next(file_it) for j in range(n)] for i in range(n)]
    
    return (np.array(B), np.array(A))


In [16]:
def test(pop_size, cro_prob, mut_prob, gen_size, off_size, runs, filename):
    runs_stats = []

    f, d = read_qap_dataset(filename)

    for i in range(runs):
        print('Run number: {}'.format(i))
        begining = time.time()
        p = geneticAgorithm(f,d,pop_size,cro_prob,mut_prob,gen_size,off_size)
        end = time.time()
        total_time = end - begining
        best = p[0]
        worst = p[-1]
        average_cost = np.mean([ind.cost for ind in p])
        runs_stats.append({ 'runtime': total_time, 'best': best, 'worst': worst, 'average_cost': average_cost })
        clear_output(wait=True)
    
    costs = [ stat['average_cost'] for stat in runs_stats ]
    times = [ stat['runtime'] for stat in runs_stats ]
 
    best_individual = min([ stat['best'] for stat in runs_stats ])
    worst_individual = max([ stat['worst'] for stat in runs_stats ])

    cost_average = np.mean(costs)
    cost_std = np.std(costs)
    
    time_average = np.mean(times)
    time_std = np.std(times)
    

    max_time = max(times)
    min_time = min(times)

    print(f'Costo promedio: {cost_average}')
    print(f'Desviación estandar de costos: {cost_std}')
    print(f'Mejor solución encontrada: {best_individual}: {best_individual.cost}')
    print(f'Peor solución encontrada: {worst_individual}: {worst_individual.cost}')
    print(f'Tiempo promedio de ejecución: {time_average}')
    print(f'Desviación estandar tiempo de ejecución: {time_std}')
    print(f'Valor máximo (tiempo de ejecución): {max_time}')
    print(f'Valor mínimo (tiempo de ejecución): {min_time}')


In [ ]:

POP_SIZE = 60
CRO_PROB = 0.9
MUT_PROB = 0.8
GEN_SIZE = 10000
OFF_SIZE = 120
RUNS = 1
FILENAME = 'lipa20a.dat'
test(POP_SIZE,CRO_PROB,MUT_PROB,GEN_SIZE,OFF_SIZE,RUNS,FILENAME)

In [ ]:
POP_SIZE = 60
CRO_PROB = 0.9
MUT_PROB = 0.8
GEN_SIZE = 10000
OFF_SIZE = 120
RUNS = 30
FILENAME = 'kra30a.dat'

test(POP_SIZE,CRO_PROB,MUT_PROB,GEN_SIZE,OFF_SIZE,RUNS,FILENAME)

In [ ]:
POP_SIZE = 60
CRO_PROB = 0.9
MUT_PROB = 0.8
GEN_SIZE = 10000
OFF_SIZE = 120
RUNS = 30
FILENAME = 'tai35b.dat'

test(POP_SIZE,CRO_PROB,MUT_PROB,GEN_SIZE,OFF_SIZE,RUNS,FILENAME)

In [ ]:
POP_SIZE = 60
CRO_PROB = 0.9
MUT_PROB = 0.8
GEN_SIZE = 10000
OFF_SIZE = 120
RUNS = 30
FILENAME = 'tho40.dat'

test(POP_SIZE,CRO_PROB,MUT_PROB,GEN_SIZE,OFF_SIZE,RUNS,FILENAME)